# PyTorch-Powered Machine Learning Project

***Import Required Libraries***

In [55]:
import pandas as pd
import numpy as np
import torch
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch.nn as nn
import torch.optim as optim

***Load and Explore the Dataset***

In [35]:
# Load dataset with proper encoding

# Upload the dataset
file_path = files.upload()

# Extract the uploaded file name
uploaded_file_name = list(file_path.keys())[0]

# Load the dataset with appropriate encoding
try:
    data = pd.read_csv(uploaded_file_name, delimiter=";", encoding="utf-8")
except UnicodeDecodeError:
    # If utf-8 fails, try 'latin-1' encoding
    data = pd.read_csv(uploaded_file_name, delimiter=";", encoding="latin-1")

# Basic info about the dataset
data.info()

Saving apartments_for_rent_classified_10K.csv to apartments_for_rent_classified_10K (4).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             10000 non-null  int64  
 1   category       10000 non-null  object 
 2   title          10000 non-null  object 
 3   body           10000 non-null  object 
 4   amenities      6451 non-null   object 
 5   bathrooms      9966 non-null   float64
 6   bedrooms       9993 non-null   float64
 7   currency       10000 non-null  object 
 8   fee            10000 non-null  object 
 9   has_photo      10000 non-null  object 
 10  pets_allowed   5837 non-null   object 
 11  price          10000 non-null  int64  
 12  price_display  10000 non-null  object 
 13  price_type     10000 non-null  object 
 14  square_feet    10000 non-null  int64  
 15  address        6673 non-null   object 
 16  cit

***Data Preprocessing***

In [61]:
# Select relevant columns and handle missing data
columns_to_use = ['bathrooms', 'bedrooms',
                  'price', 'square_feet', 'latitude', 'longitude']
# Handle missing values (drop rows with missing values)
data = data.dropna()

data = data[columns_to_use]

# Handle missing values (drop rows with missing values)
data = data.dropna()


# Split features and target
X = data.drop('square_feet', axis=1).values
y = data['square_feet'].values

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

***Create PyTorch Dataset and DataLoader***

In [62]:
class ApartmentDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create datasets
train_dataset = ApartmentDataset(X_train, y_train)
val_dataset = ApartmentDataset(X_val, y_val)
test_dataset = ApartmentDataset(X_test, y_test)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


***Define the Regression Model***

In [63]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.fc(x)

# Initialize the model
input_dim = X_train.shape[1]
model = RegressionModel(input_dim)

In [77]:
# Assuming 'model' is your defined PyTorch model
input_size = (X_train.shape[1],)  # Input size without the batch dimension
summary(model, input_size=input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 32]             192
              ReLU-2                   [-1, 32]               0
            Linear-3                   [-1, 32]           1,056
              ReLU-4                   [-1, 32]               0
            Linear-5                    [-1, 1]              33
Total params: 1,281
Trainable params: 1,281
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


***Define Loss Function and Optimizer***

In [64]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

***Training Loop***

In [65]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch).squeeze()
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Validation Loss: {val_loss/len(val_loader):.4f}")

Epoch 1/50, Train Loss: 0.4420, Validation Loss: 0.3787
Epoch 2/50, Train Loss: 0.3460, Validation Loss: 0.3701
Epoch 3/50, Train Loss: 0.3341, Validation Loss: 0.3539
Epoch 4/50, Train Loss: 0.3254, Validation Loss: 0.3559
Epoch 5/50, Train Loss: 0.3226, Validation Loss: 0.3413
Epoch 6/50, Train Loss: 0.3217, Validation Loss: 0.3477
Epoch 7/50, Train Loss: 0.3199, Validation Loss: 0.3410
Epoch 8/50, Train Loss: 0.3207, Validation Loss: 0.3398
Epoch 9/50, Train Loss: 0.3160, Validation Loss: 0.3454
Epoch 10/50, Train Loss: 0.3154, Validation Loss: 0.3414
Epoch 11/50, Train Loss: 0.3163, Validation Loss: 0.3433
Epoch 12/50, Train Loss: 0.3151, Validation Loss: 0.3444
Epoch 13/50, Train Loss: 0.3141, Validation Loss: 0.3403
Epoch 14/50, Train Loss: 0.3126, Validation Loss: 0.3500
Epoch 15/50, Train Loss: 0.3131, Validation Loss: 0.3404
Epoch 16/50, Train Loss: 0.3111, Validation Loss: 0.3412
Epoch 17/50, Train Loss: 0.3119, Validation Loss: 0.3460
Epoch 18/50, Train Loss: 0.3092, Validat

***Testing the Model***

In [67]:
# Evaluate the model with additional regression metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Collect all predictions and ground truths from the test set
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch).squeeze()
        y_true.extend(y_batch.numpy())
        y_pred.extend(outputs.numpy())

# Calculate metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)

# Print metrics
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

Mean Absolute Error (MAE): 0.3080
Mean Squared Error (MSE): 0.3158
Root Mean Squared Error (RMSE): 0.5619
R² Score: 0.6807


***Define the hyperparameter grid and tuning process***

In [70]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

In [71]:
#  Define the hyperparameter grid and tuning process
from itertools import product

# Hyperparameter grid
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]
hidden_layer_sizes = [32, 64, 128]

# Store results
results = []

# Define a function to create a DataLoader with a given batch size
def create_dataloader(X, y, batch_size):
    dataset = torch.utils.data.TensorDataset(X, y)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

# Grid search over hyperparameters
for lr, batch_size, hidden_size in product(learning_rates, batch_sizes, hidden_layer_sizes):
    print(f"Testing: LR={lr}, Batch Size={batch_size}, Hidden Size={hidden_size}")

    # Update DataLoaders with new batch size
    train_loader = create_dataloader(X_train_tensor, y_train_tensor, batch_size)
    val_loader = create_dataloader(X_val_tensor, y_val_tensor, batch_size)

    # Redefine the model
    model = nn.Sequential(
        nn.Linear(X_train_tensor.shape[1], hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, 1)
    )

    # Define loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Training loop
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            predictions = model(X_batch).squeeze()
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            predictions = model(X_batch).squeeze()
            val_loss += criterion(predictions, y_batch).item()
    val_loss /= len(val_loader)

    # Save results
    results.append((lr, batch_size, hidden_size, val_loss))
    print(f"Validation Loss: {val_loss:.4f}")

Testing: LR=0.01, Batch Size=32, Hidden Size=32
Validation Loss: 0.3416
Testing: LR=0.01, Batch Size=32, Hidden Size=64
Validation Loss: 0.3407
Testing: LR=0.01, Batch Size=32, Hidden Size=128
Validation Loss: 0.3363
Testing: LR=0.01, Batch Size=64, Hidden Size=32
Validation Loss: 0.3374
Testing: LR=0.01, Batch Size=64, Hidden Size=64
Validation Loss: 0.3369
Testing: LR=0.01, Batch Size=64, Hidden Size=128
Validation Loss: 0.3475
Testing: LR=0.01, Batch Size=128, Hidden Size=32
Validation Loss: 0.3492
Testing: LR=0.01, Batch Size=128, Hidden Size=64
Validation Loss: 0.3711
Testing: LR=0.01, Batch Size=128, Hidden Size=128
Validation Loss: 0.3466
Testing: LR=0.001, Batch Size=32, Hidden Size=32
Validation Loss: 0.3273
Testing: LR=0.001, Batch Size=32, Hidden Size=64
Validation Loss: 0.3342
Testing: LR=0.001, Batch Size=32, Hidden Size=128
Validation Loss: 0.3459
Testing: LR=0.001, Batch Size=64, Hidden Size=32
Validation Loss: 0.3340
Testing: LR=0.001, Batch Size=64, Hidden Size=64
Vali

***Select the best hyperparameters***

In [72]:
# Select the best hyperparameters
best_params = min(results, key=lambda x: x[3])  # Minimize validation loss
print(f"Best Hyperparameters: Learning Rate={best_params[0]}, Batch Size={best_params[1]}, Hidden Size={best_params[2]}")


Best Hyperparameters: Learning Rate=0.001, Batch Size=32, Hidden Size=32


***Train final model with best hyperparameters***

In [74]:
# Convert test data to PyTorch tensors
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [75]:
# Train final model with best hyperparameters
# Extract best parameters
best_lr, best_batch_size, best_hidden_size = best_params[:3]

# Update DataLoaders
train_loader = create_dataloader(X_train_tensor, y_train_tensor, best_batch_size)
test_loader = create_dataloader(X_test_tensor, y_test_tensor, best_batch_size)

# Redefine model
model = nn.Sequential(
    nn.Linear(X_train_tensor.shape[1], best_hidden_size),
    nn.ReLU(),
    nn.Linear(best_hidden_size, best_hidden_size),
    nn.ReLU(),
    nn.Linear(best_hidden_size, 1)
)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=best_lr)

# Training loop
num_epochs = 100  # Train for more epochs for the final model
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        predictions = model(X_batch).squeeze()
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()

# Evaluate the final model
model.eval()
test_loss = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        predictions = model(X_batch).squeeze()
        test_loss += criterion(predictions, y_batch).item()
test_loss /= len(test_loader)

print(f"Final Test Loss: {test_loss:.4f}")

Final Test Loss: 0.3165
